#### Objective:- Alternative script that generates points within rectangular bounds in 40min for sanity checks ( Needs to be integrated with cuSpatial for checking points within polygon) 

In [ ]:
import cudf, cupy
import pandas as pd, numpy as np
import geopandas as gpd
# from shapely.geometry import Point, Polygon
import os
import datetime
import pickle

### ETL

In [ ]:
df = pd.read_csv('data/mapped_data_full.csv',encoding='unicode_escape',dtype={'GISJOIN':'int64','ID20':'int64','STATE':'int32','COUNTY':'str','P20':'int32','P10_new':'int32'}).drop('Unnamed: 0',axis=1)
df['P_delta']=df['P20'] - df['eq_P10']
df['P_net']= df['P_delta'].apply(lambda x : 1 if x>0 else 0)
df['number'] = df.P_delta.round().abs().astype('int32')
df.head()

In [ ]:
# df =df.to_pandas()

#### MAKE function

In [ ]:
def Random_Points_in_Bounds(row):  
    polygon = row.iloc[0]
    number = row.iloc[1]
    minx, miny, maxx, maxy = polygon.bounds
    x = np.random.uniform( minx, maxx, number )
    y = np.random.uniform( miny, maxy, number )
    return [x, y]

def makeXYpair(row):
    l1 = row[0]
    l2 = row[1]
    return list(map(lambda x, y:[x,y], l1, l2))


def exec_data(state_key_list):
    c=0
    for i in state_key_list:
        c+=1
        if i< 10:
            i_str = '0'+str(i)
        else:
            i_str = str(i)
        path ='data/tl_shapefiles/tl_2021_%s_tabblock20.shp'%(i_str)
        print("started reading shape file for state ", states[i])
        if os.path.isfile(path):    
            gpdf = gpd.read_file(path)[['GEOID20', 'geometry']].sort_values('GEOID20').reset_index(drop=True)
            gpdf.GEOID20 = gpdf.GEOID20.astype('int64')
            print("completed reading shape file for state ", states[i])
            df_temp = df.query('STATE == @i')[['ID20', 'number','COUNTY','P_delta','P_net']]
            merged_df =pd.merge(gpdf,df_temp[['ID20','number']],left_on='GEOID20',right_on='ID20',how='inner')
            merged_df = merged_df[merged_df.number!=0].reset_index(drop=True)
            merged_df =merged_df.reset_index(drop=True).drop(columns=['GEOID20'])

            print("starting to generate data for "+str(states[i])+"... ")
            t1 = datetime.datetime.now()
            population_df = pd.DataFrame(merged_df[['geometry','number']].apply(Random_Points_in_Bounds,axis=1),columns=['population'])
            points_df = population_df['population'].apply(makeXYpair)
            points_df = pd.DataFrame(points_df.explode()).reset_index()
            
            pop_list =points_df['population'].to_list()
            final_df =pd.DataFrame(pop_list,columns=['x','y']).reset_index(drop=True)
            
            ids = merged_df.ID20.to_list()
            number =merged_df.number.to_list()
            
            rows = []
            for id20, n in zip(ids,number):
                rows.extend([id20]*n)
            
            
            final_df['ID20'] = pd.Series(rows)
            final_df = final_df.sort_values('ID20').reset_index(drop=True)
            final_df = pd.merge(final_df,df_temp, on='ID20',how='left')
            
            
            final_df.to_csv('data/migration_files1/migration_%s'%str(states[i])+'.csv', index=False)
            print("Processing complete for", states[i])
            print('Processing for '+str(states[i])+' complete \n total time', datetime.datetime.now() - t1)
            
            del(df_temp)
        else:
            print("shape file does not exist")
            continue

In [ ]:
# states = {1 :"AL",2 :"AK",4 :"AZ",5 :"AR",6 :"CA",8 :"CO",9 :"CT",10:"DE",11:"DC",12:"FL",13:"GA",15:"HI",
#           16:"ID",17:"IL",18:"IN",19:"IA",20:"KS",21:"KY",22:"LA",23:"ME",24:"MD",25:"MA",26:"MI",27:"MN",
#           28:"MS",29:"MO",30:"MT",31:"NE",32:"NV",33:"NH",34:"NJ",35:"NM",36:"NY",37:"NC",38:"ND",39:"OH",
#           40:"OK",41:"OR",42:"PA",44:"RI",45:"SC",46:"SD",47:"TN",48:"TX",49:"UT",50:"VT",51:"VA",53:"WA",
#           54:"WV",55:"WI",56:"WY",72:"PR"}
states= { 12:"FL",13:"GA",15:"HI",16:"ID",17:"IL",18:"IN",19:"IA",20:"KS}

In [ ]:
exec_data(states.keys())

### Concat States

In [ ]:
def merge_shape_and_states(state_key_list):
    concat_states = cudf.DataFrame()
    
    for i in state_key_list:
        if i< 10:
            i_str = '0'+str(i)
        else:
            i_str = str(i)
        path = 'data/migration_files1/migration_%s'%str(states[i])+'.csv'
        if os.path.isfile(path): 
            temp = cudf.read_csv(path,dtype={'ID20':'int64','x':'float32','y':'float32'})# Load shape files
            concat_states = cudf.concat([concat_states,temp])
        else:
            print(path)
            print("shape file does not exist")
            continue
    return concat_states

In [ ]:
indv_df = merge_shape_and_states(states.keys())
indv_df.rename(columns={'GEOID20':'ID20'},inplace=True)
indv_df.head()

### Load saved files

In [ ]:
pickle.dump(indv_df,open('fulldata_gpu_2','wb'))
# indv_df = pickle.load(open('fulldata_gpu','rb'))

In [ ]:
# indv_df = indv_df.to_pandas()

In [ ]:
indv_df = dask_cudf.from_cudf(indv_df, npartitions=2).persist()

In [ ]:
# dataset = pd.merge(indv_df,df,on='ID20',how='left')
dataset = indv_df.merge(df,on='ID20',how='left') # merge dask dfs

In [ ]:
len(dataset)

In [ ]:
del(indv_df)
del(df)

In [ ]:
dataset = dataset.sort_values('ID20')
dataset = dataset.drop(columns=['GISJOIN'])
dataset.head()

### Viz check

In [ ]:
from holoviews.element.tiles import CartoDark
import holoviews as hv
from holoviews.operation.datashader import datashade,rasterize,shade
from plotly.colors import sequential
hv.extension('plotly')

In [ ]:
dataset["easting"], dataset["northing"] = hv.Tiles.lon_lat_to_easting_northing(dataset["x"], dataset["y"])
dataset.head()

In [ ]:
dataset = hv.Dataset(dataset)

In [ ]:
mapbox_token = 'pk.eyJ1IjoibmlzaGFudGoiLCJhIjoiY2w1aXpwMXlkMDEyaDNjczBkZDVjY2l6dyJ9.7oLijsue-xOICmTqNInrBQ'
tiles= hv.Tiles().opts(mapboxstyle="dark", accesstoken=mapbox_token)
points = datashade(hv.Points(dataset, ["easting", "northing"]),cmap=sequential.Plasma)

In [ ]:
(tiles*points).opts(width=1800, height=500)